# Main Function to be called for object detection and object orientation prediction

In [ ]:
from orientation_detection.NN_module import get_prediction,load_CNN_model
from orientation_detection.render_images import get_images
from object_detection.object_detection import use_webcam,use_image,get_cropped_image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from render_module import main
from render_module.main import RenderModule
import matplotlib.pyplot as plt
import cv2
import pickle
import datetime

In [ ]:
def get_mesh(image_generator):
    return image_generator.createMesh('models/09_real_chair/model.obj', 'models/09_real_chair/texture.jpg')

### Callback Function to be sent to use_webcam function from object_detection module
- This function will get stream of cropped images from the object_detection module and run prediction using saved model and visualize the prediction 
- The webcam will keep detecting object(chair), until we hit the break or escape

In [ ]:
def run_predict_orientation(img, cropped, coords,dims,image_generator,mesh,model):
    
    print('num images',len(cropped))
    cv2.imshow('captured',img)
    
    if len(cropped) == 0:
        return
    
    #Run Prediction
    predictions = model.predict(preprocess_input(np.array(cropped).copy()))
    
    meshes = [mesh for i in range(len(cropped))]
    heights = [1 for i in range(len(cropped))]
    
    matrix = image_generator.visualize(90/180*np.pi,img, coords, dims, meshes, heights,predictions.reshape(len(predictions),2,3),True)
    cv2.imshow('Cropped',cropped[0].astype(np.uint8))
    cv2.imshow('prediction',matrix.astype(np.uint8))

In [ ]:
class Main:
    def __init__(self):
        self.image_generator=RenderModule(False,800,600)
        self.model = load_CNN_model('vectorOrientations','../trained_model/new_chair/vgg16/pretrained/2019-04-16 14:22/vgg16_model_2019-04-16 14:22')
        self.mesh = self.image_generator.createMesh('models/09_real_chair/model.obj', 'models/09_real_chair/texture.jpg')
    def callback(self,img, cropped, coords,dims):
        run_predict_orientation(img, cropped, coords,dims, self.image_generator, self.mesh, self.model)

main = Main()

#/trained_model/new_chair/comparison 19 April/VGG16/2019-04-19 04:06/vgg16_model_2019-04-19 04:06
#/trained_model/new_chair/vgg16/pretrained/2019-04-16 14:22/vgg16_model_2019-04-16 14:22

use_webcam(main.callback)